In [1]:
# Data from Weidmann and Roed 2019

rm(list = ls()) # Clear workspace

library("lme4") # For multilevel models

load("Lecture7_data.RData") # Data at the city-year level

# Getting to know the data - more here for a thorough analysis


Loading required package: Matrix



In [2]:
summary(cityyear.data)
summary(cityyear.data$year)


   geonameid           cowcode           year        nrprotests     
 Min.   :   32909   Min.   : 40.0   Min.   :2004   Min.   :  0.000  
 1st Qu.: 1033356   1st Qu.:600.0   1st Qu.:2006   1st Qu.:  0.000  
 Median : 1812491   Median :710.0   Median :2008   Median :  0.000  
 Mean   : 3241959   Mean   :627.8   Mean   :2008   Mean   :  0.549  
 3rd Qu.: 7001367   3rd Qu.:710.0   3rd Qu.:2010   3rd Qu.:  0.000  
 Max.   :11054823   Max.   :830.0   Max.   :2012   Max.   :163.000  
                                                                    
  nrprotests_l      lspop2008_5k_centered nightlights_5k_pc_l_centered
 Min.   :  0.0000   Min.   :-2.741798     Min.   :-2.4474788          
 1st Qu.:  0.0000   1st Qu.:-0.444120     1st Qu.:-0.3323680          
 Median :  0.0000   Median :-0.131991     Median : 0.0834102          
 Mean   :  0.5141   Mean   :-0.000827     Mean   :-0.0004899          
 3rd Qu.:  0.0000   3rd Qu.: 0.405967     3rd Qu.: 0.4067887          
 Max.   :163.0000   Ma

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   2004    2006    2008    2008    2010    2012 

In [3]:
ncities <- unique(cityyear.data$geonameid)
ncountries <- unique(cityyear.data$cowcode)

In [7]:
length(ncities)

[1] 1564

In [8]:
length(ncountries)

[1] 61

In [11]:
head(cityyear.data)

,geonameid,cowcode,year,nrprotests,nrprotests_l,lspop2008_5k_centered,nightlights_5k_pc_l_centered
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,32909,630,2004,0,NA,-0.5675273,0.8041611
2,32909,630,2005,0,0,-0.5675273,0.8392026
3,32909,630,2006,0,0,-0.5675273,0.8539909
4,32909,630,2007,0,0,-0.5675273,0.8339887
5,32909,630,2008,0,0,-0.5675273,0.8273262
6,32909,630,2009,0,0,-0.5675273,0.8674713


In [12]:
# Complete pooling: no group modelling
m1 <- lm(nrprotests ~ nightlights_5k_pc_l_centered, data=cityyear.data)
summary(m1)


Call:
lm(formula = nrprotests ~ nightlights_5k_pc_l_centered, data = cityyear.data)

Residuals:
    Min      1Q  Median      3Q     Max 
 -1.639  -0.661  -0.479  -0.314 162.092 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                   0.54875    0.03404  16.121  < 2e-16 ***
nightlights_5k_pc_l_centered -0.44532    0.05703  -7.809 6.18e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.995 on 13773 degrees of freedom
Multiple R-squared:  0.004408,	Adjusted R-squared:  0.004336 
F-statistic: 60.98 on 1 and 13773 DF,  p-value: 6.181e-15


In [13]:
# No pooling: fixed effects
m2 <- lm(nrprotests ~ nightlights_5k_pc_l_centered + factor(geonameid) - 1, data=cityyear.data)

In [19]:
options(max.print=75) # To reduce the output in the model summaries below
summary(m2)


Call:
lm(formula = nrprotests ~ nightlights_5k_pc_l_centered + factor(geonameid) - 
    1, data = cityyear.data)

Residuals:
    Min      1Q  Median      3Q     Max 
-33.332  -0.071  -0.003   0.025 136.886 

Coefficients:
                               Estimate Std. Error t value Pr(>|t|)    
nightlights_5k_pc_l_centered  5.258e-01  1.982e-01   2.653 0.007988 ** 
factor(geonameid)32909       -4.502e-01  9.328e-01  -0.483 0.629386    
factor(geonameid)70225        9.761e+00  9.250e-01  10.553  < 2e-16 ***
factor(geonameid)71137        2.654e+01  9.312e-01  28.497  < 2e-16 ***
factor(geonameid)74477        2.403e+00  9.198e-01   2.613 0.008999 ** 
factor(geonameid)76482        4.187e-01  9.307e-01   0.450 0.652806    
factor(geonameid)77940       -1.308e-01  9.186e-01  -0.142 0.886784    
factor(geonameid)78137       -1.132e-01  9.182e-01  -0.123 0.901905    
factor(geonameid)78931       -1.417e-01  9.188e-01  -0.154 0.877435    
factor(geonameid)79415        2.328e+00  9.172e-01   2.53

In [22]:
# Partial pooling: varying-intercept
m3 <- lmer(nrprotests ~ nightlights_5k_pc_l_centered + (1 | geonameid), data=cityyear.data)
summary(m3) # Note that std.dev of geonameid (Intercept) is large compared to the residual standard error
# Std.dev is larger between "groups" than between "individuals"

Linear mixed model fit by REML ['lmerMod']
Formula: nrprotests ~ nightlights_5k_pc_l_centered + (1 | geonameid)
   Data: cityyear.data

REML criterion at convergence: 71050.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-10.340  -0.029  -0.019  -0.012  50.291 

Random effects:
 Groups    Name        Variance Std.Dev.
 geonameid (Intercept) 10.23    3.199   
 Residual               7.62    2.760   
Number of obs: 13775, groups:  geonameid, 1564

Fixed effects:
                             Estimate Std. Error t value
(Intercept)                   0.61519    0.08429   7.298
nightlights_5k_pc_l_centered -0.22638    0.11669  -1.940

Correlation of Fixed Effects:
            (Intr)
nghtlg_5___ 0.010 

In [23]:
coef(m3) # Varying-intercepts and beta coefficient

$geonameid
        (Intercept) nightlights_5k_pc_l_centered
32909   0.226004834                   -0.2263751
70225   8.643732360                   -0.2263751
71137  23.993051357                   -0.2263751
74477   2.027575636                   -0.2263751
76482  -0.119471254                   -0.2263751
77940   0.099012341                   -0.2263751
78137   0.092012591                   -0.2263751
78931   0.103353340                   -0.2263751
79415   2.204303850                   -0.2263751
81302   0.251080473                   -0.2263751
88319   4.130299648                   -0.2263751
100425  0.180283173                   -0.2263751
101628  0.079232897                   -0.2263751
101760 -0.004537224                   -0.2263751
104515  0.129910578                   -0.2263751
105072  0.117552966                   -0.2263751
105343 -0.014031928                   -0.2263751
106281  0.143998198                   -0.2263751
107304  0.155435994                   -0.2263751
107968  0

In [24]:
ranef(m3) # Change in intercepts for each group

$geonameid
       (Intercept)
32909  -0.38918325
70225   8.02854428
71137  23.37786327
74477   1.41238755
76482  -0.73465934
77940  -0.51617574
78137  -0.52317549
78931  -0.51183474
79415   1.58911577
81302  -0.36410761
88319   3.51511157
100425 -0.43490491
101628 -0.53595518
101760 -0.61972531
104515 -0.48527750
105072 -0.49763512
105343 -0.62922001
106281 -0.47118988
107304 -0.45975209
107968 -0.58234214
108410  0.82051060
109101 -0.52330479
109223 -0.67348200
109323 -0.49838990
109571 -0.26969847
110336 -0.63145021
111453  0.84700460
111822 -0.23643947
112931 27.12378605
113514 -0.66971882
113646  2.41657826
115019  0.11298176
116996 -0.22237128
117574  0.35919239
118743  1.00816267
119208  0.30380357
119505  1.00957663
121801  1.20278639
124665  0.69726053
127349 -0.39420011
128226 -0.03898873
128234 -0.32899048
128747 -0.59802479
132144 -0.17428164
132892 -0.43844630
140044 -0.36845348
140463 -0.49323439
141681 -0.10652938
143083  0.83277003
143127 -0.51066872
144448  0.77287929
1

In [26]:
# Partial pooling: varying-intercept
# We can try to reduce between unit heterogeneity by including other variables
m3.2 <- lmer(nrprotests ~ nightlights_5k_pc_l_centered + lspop2008_5k_centered + nrprotests_l + (1 | geonameid), data=cityyear.data)
summary(m3.2) # Note that std.dev of geonameid (Intercept) is large compared to the residual standard error
# Std.dev is larger between "groups" than between "individuals"

Linear mixed model fit by REML ['lmerMod']
Formula: nrprotests ~ nightlights_5k_pc_l_centered + lspop2008_5k_centered +  
    nrprotests_l + (1 | geonameid)
   Data: cityyear.data

REML criterion at convergence: 62633

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-11.813  -0.062  -0.032  -0.014  50.454 

Random effects:
 Groups    Name        Variance Std.Dev.
 geonameid (Intercept) 3.186    1.785   
 Residual              8.273    2.876   
Number of obs: 12211, groups:  geonameid, 1564

Fixed effects:
                             Estimate Std. Error t value
(Intercept)                  0.403109   0.052430   7.689
nightlights_5k_pc_l_centered 0.010564   0.096429   0.110
lspop2008_5k_centered        0.502319   0.087543   5.738
nrprotests_l                 0.404511   0.008529  47.425

Correlation of Fixed Effects:
            (Intr) n_5___ l2008_
nghtlg_5___ -0.011              
lspp2008_5_  0.002  0.506       
nrprotsts_l -0.092  0.007 -0.066

In [27]:
coef(m3.2) # Varying-intercepts and beta coefficients

$geonameid
       (Intercept) nightlights_5k_pc_l_centered lspop2008_5k_centered
32909   0.30712362                   0.01056388              0.502319
70225   5.05178175                   0.01056388              0.502319
71137  14.50092254                   0.01056388              0.502319
74477   1.38902385                   0.01056388              0.502319
76482  -0.04537984                   0.01056388              0.502319
77940   0.21934957                   0.01056388              0.502319
78137   0.23219070                   0.01056388              0.502319
78931   0.22956945                   0.01056388              0.502319
79415   1.29730332                   0.01056388              0.502319
81302   0.36951282                   0.01056388              0.502319
88319   3.13534235                   0.01056388              0.502319
100425  0.31064221                   0.01056388              0.502319
101628  0.05848927                   0.01056388              0.502319
101760 -0

In [28]:
ranef(m3.2) # Change in intercepts for each group

$geonameid
        (Intercept)
32909  -0.095985229
70225   4.648672902
71137  14.097813695
74477   0.985915004
76482  -0.448488685
77940  -0.183759280
78137  -0.170918142
78931  -0.173539392
79415   0.894194470
81302  -0.033596024
88319   2.732233499
100425 -0.092466639
101628 -0.344619579
101760 -0.499028948
104515 -0.488091138
105072 -0.246101323
105343 -0.760606148
106281 -0.221485195
107304 -0.192858010
107968 -0.409045189
108410  0.232748536
109101 -0.312079216
109223 -0.603993961
109323 -0.233618922
109571 -0.105756097
110336 -0.531124934
111453  0.525350552
111822 -0.052630059
112931 13.270274802
113514 -0.613537331
113646  0.815783126
115019 -0.093760875
116996  0.025700449
117574  0.236066674
118743  0.526649072
119208 -0.033649491
119505  0.539197168
121801  0.621589964
124665  0.232275817
127349 -0.238822096
128226 -0.052482639
128234 -0.236413708
128747 -0.478778300
132144 -0.027056572
132892 -0.086760718
140044 -0.063406868
140463 -0.172189777
141681  0.002045624
143083  0

In [29]:
# Partial pooling: varying-intercept by multiple groups
m4 <- lmer(nrprotests ~ nightlights_5k_pc_l_centered + (1 | geonameid) + (1 | cowcode), data=cityyear.data)
summary(m4)

Linear mixed model fit by REML ['lmerMod']
Formula: nrprotests ~ nightlights_5k_pc_l_centered + (1 | geonameid) +  
    (1 | cowcode)
   Data: cityyear.data

REML criterion at convergence: 70943.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-10.112  -0.037  -0.012   0.002  50.268 

Random effects:
 Groups    Name        Variance Std.Dev.
 geonameid (Intercept) 8.884    2.981   
 cowcode   (Intercept) 4.282    2.069   
 Residual              7.622    2.761   
Number of obs: 13775, groups:  geonameid, 1564; cowcode, 61

Fixed effects:
                             Estimate Std. Error t value
(Intercept)                    1.4250     0.3075   4.634
nightlights_5k_pc_l_centered  -0.3471     0.1300  -2.670

Correlation of Fixed Effects:
            (Intr)
nghtlg_5___ 0.060 

In [30]:
coef(m4) # Varying-intercepts and beta coefficients

$geonameid
       (Intercept) nightlights_5k_pc_l_centered
32909   0.11078622                   -0.3470979
70225   6.05969211                   -0.3470979
71137  21.20966160                   -0.3470979
74477  -0.45192043                   -0.3470979
76482  -2.62419773                   -0.3470979
77940  -2.29304772                   -0.3470979
78137  -2.30365706                   -0.3470979
78931  -2.28646818                   -0.3470979
79415  -0.23815533                   -0.3470979
81302  -0.04959358                   -0.3470979
88319   3.77434085                   -0.3470979
100425  1.34024216                   -0.3470979
101628  1.18708284                   -0.3470979
101760  1.06011460                   -0.3470979
104515  1.15569902                   -0.3470979
105072  1.24516359                   -0.3470979
105343  0.99162638                   -0.3470979
106281  1.28524595                   -0.3470979
107304  1.30258193                   -0.3470979
107968  1.11677531           

In [31]:
ranef(m4) # Change in intercepts for each group

$geonameid
        (Intercept)
32909  -1.314196097
70225   4.634709795
71137  19.784679280
74477  -1.876902745
76482  -4.049180045
77940  -3.718030033
78137  -3.728639376
78931  -3.711450492
79415  -1.663137644
81302  -1.474575900
88319   2.349358533
100425 -0.084740153
101628 -0.237899478
101760 -0.364867714
104515 -0.269283300
105072 -0.179818728
105343 -0.433355938
106281 -0.139736364
107304 -0.122400388
107968 -0.308207001
108410  1.006600930
109101 -0.218725591
109223 -0.446345364
109323 -0.180962738
109571  0.057464338
110336 -0.382638849
111453 -0.143803202
111822 -1.190880945
112931 25.672031763
113514 -1.739396700
113646  1.315504694
115019 -0.931758827
116996 -1.169558146
117574 -0.612680999
118743 -0.007734716
119208 -0.750729760
119505 -0.005591604
121801  0.179056802
124665 -0.370766459
127349 -1.375897365
128226 -1.053901931
128234 -1.331158138
128747 -1.630731892
132144 -1.150767243
132892 -1.388862855
140044 -1.336873866
140463 -1.471903751
141681 -1.102174224
143083 -0

In [32]:
# Partial pooling: varying-intercept and varying slope
m5 <- lmer(nrprotests ~ nightlights_5k_pc_l_centered + (1 + nightlights_5k_pc_l_centered | geonameid), data=cityyear.data)
summary(m5) 

boundary (singular) fit: see ?isSingular



Linear mixed model fit by REML ['lmerMod']
Formula: 
nrprotests ~ nightlights_5k_pc_l_centered + (1 + nightlights_5k_pc_l_centered |  
    geonameid)
   Data: cityyear.data

REML criterion at convergence: 70478.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-10.046  -0.063  -0.028  -0.011  49.733 

Random effects:
 Groups    Name                         Variance Std.Dev. Corr 
 geonameid (Intercept)                  7.520    2.742         
           nightlights_5k_pc_l_centered 7.208    2.685    -1.00
 Residual                               7.680    2.771         
Number of obs: 13775, groups:  geonameid, 1564

Fixed effects:
                             Estimate Std. Error t value
(Intercept)                   0.51386    0.07733   6.645
nightlights_5k_pc_l_centered -0.14233    0.11135  -1.278

Correlation of Fixed Effects:
            (Intr)
nghtlg_5___ -0.740
convergence code: 0
boundary (singular) fit: see ?isSingular


In [33]:
# What does the std.dev of nightlights_5k_pc_l_centered mean?
coef(m5) # Varying-intercepts and beta coefficients

$geonameid
        (Intercept) nightlights_5k_pc_l_centered
32909   0.061661769                   0.30040184
70225   5.283407325                  -4.81203219
71137  14.229668906                 -13.57101365
74477   1.346729779                  -0.95776467
76482   0.174050117                   0.19036622
77940  -0.007630516                   0.36824357
78137   0.001457121                   0.35934617
78931  -0.014756578                   0.37522046
79415   1.940293961                  -1.53890324
81302   0.064404859                   0.29771618
88319   5.452842492                  -4.97792043
100425 -0.107627693                   0.46614742
101628  0.013460648                   0.34759393
101760  0.101983679                   0.26092403
104515  0.285949816                   0.08080902
105072 -0.041216145                   0.40112612
105343  0.232902774                   0.13274558
106281 -0.080157379                   0.43925216
107304 -0.094753867                   0.45354309
107968  0

In [34]:
ranef(m5) # Change in intercepts for each group

$geonameid
       (Intercept) nightlights_5k_pc_l_centered
32909   -0.4521975                    0.4427312
70225    4.7695481                   -4.6697028
71137   13.7158096                  -13.4286842
74477    0.8328705                   -0.8154353
76482   -0.3398092                    0.3326956
77940   -0.5214898                    0.5105730
78137   -0.5124022                    0.5016756
78931   -0.5286159                    0.5175499
79415    1.4264347                   -1.3965738
81302   -0.4494544                    0.4400456
88319    4.9389832                   -4.8355910
100425  -0.6214870                    0.6084768
101628  -0.5003986                    0.4899233
101760  -0.4118756                    0.4032534
104515  -0.2279095                    0.2231384
105072  -0.5550754                    0.5434555
105343  -0.2809565                    0.2750750
106281  -0.5940167                    0.5815816
107304  -0.6086131                    0.5958725
107968  -0.4463299           

In [35]:
# Decomposing between and within effects with a partial pooling (varying-intercept) model

library(plyr)

mean <- ddply(cityyear.data, .(geonameid), 
                    summarise, 
                    between_nightlights_5k_pc_l_centered=mean(nightlights_5k_pc_l_centered))

# Merging with data

cityyear.data <- merge(cityyear.data,mean,by.x=c("geonameid"),by.y=c("geonameid"), all.x=T)

#cityyear.data$within_nightlights_5k_pc_l_centered <- cityyear.data$nightlights_5k_pc_l_centered - cityyear.data$between_nightlights_5k_pc_l_centered

m6 <- lmer(nrprotests ~ nightlights_5k_pc_l_centered + between_nightlights_5k_pc_l_centered + (1 | geonameid), data=cityyear.data)
summary(m6) # Note that std.dev of country (Intercept) is large compared to the residual standard error
coef(m6) # Varying-intercepts and beta coefficients
ranef(m6) # Change in intercepts for each group

cor(cityyear.data$nightlights_5k_pc_l_centered, cityyear.data$between_nightlights_5k_pc_l_centered, method="pearson") # Correlation between the two is HIGH


Linear mixed model fit by REML ['lmerMod']
Formula: 
nrprotests ~ nightlights_5k_pc_l_centered + between_nightlights_5k_pc_l_centered +  
    (1 | geonameid)
   Data: cityyear.data

REML criterion at convergence: 71029.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-10.341  -0.041  -0.019  -0.003  50.313 

Random effects:
 Groups    Name        Variance Std.Dev.
 geonameid (Intercept) 10.187   3.192   
 Residual               7.611   2.759   
Number of obs: 13775, groups:  geonameid, 1564

Fixed effects:
                                     Estimate Std. Error t value
(Intercept)                           0.61232    0.08412   7.279
nightlights_5k_pc_l_centered          0.52584    0.19871   2.646
between_nightlights_5k_pc_l_centered -1.14736    0.24529  -4.678

Correlation of Fixed Effects:
            (Intr) n_5___
nghtlg_5___  0.000       
btwn_n_5___  0.007 -0.810

$geonameid
        (Intercept) nightlights_5k_pc_l_centered
32909   0.538249475                    0.5258361
70225   8.421594677                    0.5258361
71137  23.690733830                    0.5258361
74477   1.901627845                    0.5258361
76482  -0.410037195                    0.5258361
77940   0.189671693                    0.5258361
78137   0.170458295                    0.5258361
78931   0.201587168                    0.5258361
79415   2.207638783                    0.5258361
81302   0.427993933                    0.5258361
88319   4.270717270                    0.5258361
100425  0.412749496                    0.5258361
101628  0.135379709                    0.5258361
101760 -0.094558317                    0.5258361
104515 -0.083686469                    0.5258361
105072  0.240563284                    0.5258361
105343 -0.299704926                    0.5258361
106281  0.313151986                    0.5258361
107304  0.344547241                    0.5258361
107968  0

$geonameid
        (Intercept)
32909  -0.074074865
70225   7.809270337
71137  23.078409490
74477   1.289303505
76482  -1.022361535
77940  -0.422652647
78137  -0.441866045
78931  -0.410737172
79415   1.595314443
81302  -0.184330407
88319   3.658392930
100425 -0.199574845
101628 -0.476944631
101760 -0.706882657
104515 -0.696010809
105072 -0.371761056
105343 -0.912029266
106281 -0.299172354
107304 -0.267777099
107968 -0.604270752
108410  0.560103138
109101 -0.442220942
109223 -0.854437750
109323 -0.373832846
109571 -0.104274569
110336 -0.739065980
111453  0.990995479
111822 -0.012982967
112931 26.734336545
113514 -0.844108299
113646  2.254826724
115019  0.050708145
116996  0.025632378
117574  0.547439229
118743  1.075183520
119208  0.216319283
119505  1.079064670
121801  1.251230392
124665  0.579967613
127349 -0.266930374
128226 -0.008261189
128234 -0.267023361
128747 -0.647317575
132144 -0.021452727
132892 -0.209295511
140044 -0.196259263
140463 -0.359681625
141681 -0.014566542
143083  0

[1] 0.980167